In [1]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from modules import solver
from pyballistics import ozvb_lagrange, get_db_powder, get_powder_names
from tqdm.notebook import tqdm, trange

In [2]:
print(solver.__doc__)

 
version 1.0

This module solves inverse problem of internal ballistics.




## Исходные данные
задаются начальные данные по орудию и формируется словарь

__Основные парметры__

In [3]:
d = 125*1e-3          # калибр м
q = 4.85           # вес снаряда кг
velocity_pm = 1700         # дульная скорость снаряда
n_s = 1           # нарезное орудие
max_pressure = 1000*1e6    # максимальное давление Па
tube_lenght = 5543*10e-3      # длина трубы  

__Дополнительные параметры__

In [4]:
# !! fix it
p_fors = 10*1e6      #давление форсирования (гладкоствольные 7-15 МПа)

#нормализация (в данный момент не используется)
norm_ro = 1000 
norm_w = 1

__Словарь__

In [5]:
initial_dict_static =  {
   'powders': 
    [
       {'omega': None, 'dbname': None}
    ],
  'init_conditions': 
    {
       'q': q,
       'd': d,
       'W_0': None ,
       'phi_1': 1.0,
       'p_0': p_fors, 
       'n_S': 1
    },
  'igniter': 
    {
       'p_ign_0': 5000000.0 #check it 
    },
  
  'meta_lagrange': 
    {
       'CFL': 0.9, 
       'n_cells': 300
    },
  'stop_conditions': 
    {
       'x_p': tube_lenght,
       'steps_max': 8000,   
       't_max': 0.05,
       'p_max': max_pressure,
        'v_p': velocity_pm,
    }
 }

## Решение обратной задачи

__Инициализация массивов под решение__

In [6]:
powders_names = get_powder_names()
n = len(powders_names)
wq_0 = 0.9
ro_0 = 500

In [7]:
optim_parametrs = list()

__Цикл решения__:

    1. Внешний цикл по порохам
    2. решение церез штафную функцию

In [ ]:
%%time

for i in trange(len(powders_names)):
    initial_dict = copy.deepcopy(initial_dict_static)
    initial_dict['powders'][0]['dbname'] = powders_names[i]
    initial_dict['powders'][0]['omega'] = wq_0*q
    initial_dict['init_conditions']['W_0'] = wq_0*q/ro_0
    try:
        solv = solver.Solver(initial_dict)
        solv.solution()
    except(ValueError, TypeError):
        optim_parametrs.append(None)
        continue
    else:
        optim_parametrs.append(solv.get_solution())


  0%|          | 0/102 [00:00<?, ?it/s]

No success in result
No success in result
No success in result


__Выбор лучшего решения__